# <font color=4CBB17>**PROYECTO KAGGLE**</font>

<font color=FF7F50><h3><b>Estudiante:</b></font> Mateo Toro López</h3>
<font color=FF7F50><h3><b>Programa:</b></font> Ingeniería Industrial</h3>


### <font color=1E90FF>**Descripción**</font>

Las **Pruebas Saber Pro** son exámenes estandarizados que se administran en Colombia para evaluar la calidad y el nivel de conocimiento y competencias de los estudiantes de educación superior, es decir, de instituciones de educación superior como universidades y tecnológicos. Estas pruebas son parte de los esfuerzos del **Gobierno de Colombia** para monitorear y mejorar la calidad de la educación superior en el país.

Estas Pruebas constan de cinco componentes genéricos: **Inglés, Lectura Crítica, Competencias Ciudadanas, Razonamiento Cuantitativo y Comunicación Escrita**.

El trabajo será crear un modelo de clasificación que, para cada estudiante, prediga qué desempeño va a tener: **bajo, medio-bajo, medio-alto o alto**.


## <font color=4CBB17>**Librerias**</font>


In [1]:
# Datos
import pandas as pd
import numpy as np
from itertools import product

## Visualización
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

## Modelado

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import statsmodels.api as sm
from sklearn.metrics import classification_report
import math


from collections import Counter
import time

# Para modelo Random Forest
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score



# <font color=4CBB17>**Descargar la base de datos desde Kaggle**</font>


### <font color=1E90FF>**Instrucciones**</font>

**1.** Crea un archivo kaggle.json con tu token de autenticación (en kaggle → haz clic en el ícono de usuario en la esquina superior derecha → configuración → API crear nuevo token)

**2.** Subir el archivo kaggle.json a este espacio de trabajo de Colab ⬇️

In [2]:
#Subir el archivo kaggle.json a este espacio de trabajo de Colab
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


**3.** Correr la sigiente celda ⬇️

In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

 84% 17.0M/20.1M [00:00<00:00, 176MB/s]
100% 20.1M/20.1M [00:00<00:00, 189MB/s]


### <font color=1E90FF>**Descomprimir e inspeccionar datos**</font>


In [4]:
!unzip udea*.zip > /dev/null

In [5]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  50135751 test.csv
   692501  10666231 118025055 train.csv
  1286075  15528571 172877479 total


### <font color=1E90FF>**cargar `train.csv` con pandas**</font>


In [6]:
# Librerias para procesamiento
import pandas as pd
import numpy as np

df = pd.read_csv("train.csv")

In [7]:
# Cuantas filas y columnas tiene el dataframe
rows, columns = df.shape
print(f"Número de filas: {rows}")
print(f"Número de columnas: {columns}")

Número de filas: 692500
Número de columnas: 12


In [8]:
df.set_index('ID', inplace=True)

In [70]:
df1 = df.copy() # Se crea una copia del dataframe original


# <font color=4CBB17>**Preprocesado**</font>


In [71]:
df1.PERIODO.value_counts()

,count
PERIODO,
20195,180873
20203,171838
20212,171412
20183,164818
20194,1472
20213,1178
20202,490
20184,254
20196,165


In [72]:
# Se crea una nueva columna llamada AÑO ya que hay unas categorias muy grandes y las otras demasiado pequeñas
df1['PERIODO'] = df1['PERIODO'].astype(str)
df1['AÑO'] = ['2018' if i in ['20183', '20184']
              else '2019' if i in ['20195', '20194', '20196']
              else '2020' if i in ['20203', '20202']
              else '2021' if i in ['20212', '20213']
              else i for i in df1['PERIODO']]

In [73]:
df1.AÑO.value_counts()

,count
AÑO,
2019,182510
2021,172590
2020,172328
2018,165072


>La variable PERIODO no tenía datos nulos, por lo tanto no fue necesario reemplazarlos, se creo la nueva variable AÑO ya que los periodos no estaban completos

In [74]:
# Asignar a los valores nulos "No info" para variables categoricas educación de la madre y padre

df1.FAMI_EDUCACIONMADRE.values[df1.FAMI_EDUCACIONMADRE.isna()] = 'No info'

df1.FAMI_EDUCACIONPADRE.values[df1.FAMI_EDUCACIONPADRE.isna()] = 'No info'


sum(df1.FAMI_EDUCACIONMADRE.isna()), sum(df1.FAMI_EDUCACIONPADRE.isna())

(0, 0)

In [75]:
# Se reeemplazan las categorias 'No sabe', 'No Aplica', 'No info' en una nueva llamada indeterminado
df1['FAMI_EDUCACIONMADRE'] = ['Indeterminado' if i in ['No sabe', 'No Aplica', 'No info'] else i for i in df1['FAMI_EDUCACIONMADRE'].values]
df1['FAMI_EDUCACIONPADRE'] = ['Indeterminado' if i in ['No sabe', 'No Aplica', 'No info'] else i for i in df1['FAMI_EDUCACIONPADRE'].values]

In [76]:
df1.FAMI_EDUCACIONPADRE.value_counts()

,count
FAMI_EDUCACIONPADRE,
Secundaria (Bachillerato) completa,128289
Primaria incompleta,125675
Educación profesional completa,83117
Secundaria (Bachillerato) incompleta,71654
Técnica o tecnológica completa,62995
Primaria completa,55958
Indeterminado,48999
Postgrado,44169
Educación profesional incompleta,27084


In [77]:
df1.FAMI_EDUCACIONMADRE.value_counts()

,count
FAMI_EDUCACIONMADRE,
Secundaria (Bachillerato) completa,141744
Primaria incompleta,99420
Técnica o tecnológica completa,89542
Educación profesional completa,85326
Secundaria (Bachillerato) incompleta,81012
Primaria completa,56125
Postgrado,46246
Indeterminado,28599
Técnica o tecnológica incompleta,27533


In [78]:
# Asignar a los valores nulos "No info" en la variable del Valor matricula Universidad
df1.ESTU_VALORMATRICULAUNIVERSIDAD.values[df1.ESTU_VALORMATRICULAUNIVERSIDAD.isna()] = 'No info'


In [79]:
# Se asigna el promedio de pago de matricula para cada categoria dentro de la variable

cmap = {'Entre 1 millón y menos de 2.5 millones': 1.75,
    'Entre 2.5 millones y menos de 4 millones': 3.25,
    'Menos de 500 mil': .250,
    'Entre 500 mil y menos de 1 millón': .75,
    'Entre 4 millones y menos de 5.5 millones': 4.75,
    'Más de 7 millones': 7.75,
    'Entre 5.5 millones y menos de 7 millones': 6.25,
    'No pagó matrícula': 0,
    'No info': -1}

# Usar map para transformar los valores
df1['ESTU_VALORMATRICULAUNIVERSIDAD'] = df1['ESTU_VALORMATRICULAUNIVERSIDAD'].map(cmap)

# Contar los valores únicos
df1['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts()

,count
ESTU_VALORMATRICULAUNIVERSIDAD,
1.75,204048
3.25,127430
0.25,80263
0.75,78704
4.75,69736
7.75,68014
6.25,38490
0.00,19528
-1.00,6287


In [80]:
counts = df1['ESTU_PRGM_DEPARTAMENTO'].value_counts()

umbral = 2000
categorias_a_mantener = counts[counts > umbral].index

# Agrupar las categorías menos frecuentes como "Otros"
df1['ESTU_PRGM_DEPARTAMENTO'] = df1['ESTU_PRGM_DEPARTAMENTO'].apply(lambda x: x if x in categorias_a_mantener else 'Zonas dificil acceso')


In [81]:
df1['ESTU_PRGM_DEPARTAMENTO'].value_counts()

,count
ESTU_PRGM_DEPARTAMENTO,
BOGOTÁ,282159
ANTIOQUIA,83607
VALLE,44588
ATLANTICO,41020
SANTANDER,28828
NORTE SANTANDER,22588
BOLIVAR,20629
BOYACA,14048
CUNDINAMARCA,14018


In [82]:
# Se verifica nuevamente nulos en la variable ESTU_HORASSEMANATRABAJA
df1.ESTU_HORASSEMANATRABAJA.isna().sum()

30857

In [83]:
## Asignar a los valores nulos "No info" para variable de horas que trabaja el estudiante a la semana
df1.ESTU_HORASSEMANATRABAJA.values[df1.ESTU_HORASSEMANATRABAJA.isna()] = 'No info'
sum(df1.ESTU_HORASSEMANATRABAJA.isna())

0

In [84]:
# Se asigna el promedio de horas para cada categoria dentro de la variable

emap = {'0': 0,
    'Menos de 10 horas': 5,
    'Entre 11 y 20 horas': 15.5,
    'Entre 21 y 30 horas': 25.5,
    'Más de 30 horas': 35.5,
    'No info': -1}

# Usar map para transformar los valores
df1['ESTU_HORASSEMANATRABAJA'] = df1['ESTU_HORASSEMANATRABAJA'].map(emap)

# Contar los valores únicos
df1['ESTU_HORASSEMANATRABAJA'].value_counts()

,count
ESTU_HORASSEMANATRABAJA,
35.5,249352
0.0,116550
15.5,115857
25.5,92693
5.0,87191
-1.0,30857


In [85]:
df1['ESTU_HORASSEMANATRABAJA'].value_counts()

,count
ESTU_HORASSEMANATRABAJA,
35.5,249352
0.0,116550
15.5,115857
25.5,92693
5.0,87191
-1.0,30857


In [86]:
df1.FAMI_ESTRATOVIVIENDA.values[df.FAMI_ESTRATOVIVIENDA.isna()] = 'No info'
sum(df1.FAMI_ESTRATOVIVIENDA.isna())

0

In [87]:
df1['FAMI_ESTRATOVIVIENDA'].value_counts()

,count
FAMI_ESTRATOVIVIENDA,
Estrato 2,232671
Estrato 3,210685
Estrato 1,111991
Estrato 4,65514
No info,32137
Estrato 5,23608
Estrato 6,12605
Sin Estrato,3289


In [88]:
df1['ESTU_PRGM_ACADEMICO'].value_counts()


,count
ESTU_PRGM_ACADEMICO,
DERECHO,53244
ADMINISTRACION DE EMPRESAS,51902
CONTADURIA PUBLICA,39664
PSICOLOGIA,31932
INGENIERIA INDUSTRIAL,28481
...,...
LICENCIATURA EN ESPAÑOL Y FILOLOGÍA,1
ADMINISTRACION EN NEGOCIOS INTERNACIONALES,1
ADMINISTRACION DE COMERCIO EXTERIOR,1


In [89]:
# Mapeo de carreras
categoria_map = {
    'Ingeniería Industrial': ['INGENIERIA INDUSTRIAL', 'INGENIERÍA INDUSTRIAL'],
    'Otras Ingenierías': ['INGENIERIA', 'INGENIERÍA', 'INGE'],
    'Licenciaturas': ['LICENCIATURA',],
    'Exactas': ['QUIMICA', 'ESTAD', 'BIOLOG', 'ANTRO', 'MATEMATICAS', 'FISICA', 'GEOLO'],
    'Lenguas':['INGLES', 'LITER', 'LENGUA', 'IDIOMA'],
    'Ciencias Economicas':['CONTADURÍA', 'CONTADURIA', 'ECONOM','FINAN'],
    'Administracion de empresas': ['ADMINISTRACION DE EMPRESAS', 'ADMINISTRACIÓN DE EMPRESAS', 'ADMINISTRACION DE NEGOCIOS INTERNACIONALES'],
    'Otras Admin':  ['ADMINISTRACION', 'ADMINISTRACIÓN'],
    'Animales': ['ZOOTECNIA', 'VETERINARIA'],
    'Salud': ['ENFERM', 'ODONTO','SALUD', 'QUIR', 'FISIO', 'NUTRI', 'FONOAUDIO' ],
    'Ciencias Sociales': ['PSICO', 'COMUNICA','SOCIAL','PERIO','HIST', 'SOCIO', 'FILOSO', 'TEOLO'],
    'Negocios': ['NEGOCIOS', 'MERCADEO', 'MARKETING', 'PUBLICIDAD', 'INTERNAC'],
    'Derecho': ['DERECHO'],
    'Ciencias Politicas':['CIENCIA POLITICA', 'POLITICA','POLÍTICA', 'JURISPRUDENCIA'],
    'Medicina':['MEDICINA'],
    'Arquitectura y Diseño':['ARQUITECTURA', 'DISEÑO'],
    'Entretenimiento':['ARTE', 'ARTES', 'MUSICA','MÚSICA' 'CINE', 'TV','TEATRO']
}

# Clasificación de las carreras usando apply y búsqueda de key words
df1['PRGM_AREA'] = df1['ESTU_PRGM_ACADEMICO'].apply(lambda carrera:
    next((categoria for categoria, keywords in categoria_map.items()
          if any(keyword in carrera.upper() for keyword in keywords)),
         'Otras'))

In [90]:
df1['PRGM_AREA'].value_counts()

,count
PRGM_AREA,
Otras Ingenierías,113640
Ciencias Sociales,85293
Administracion de empresas,84790
Ciencias Economicas,72707
Licenciaturas,66280
Derecho,53545
Otras Admin,35016
Ingeniería Industrial,34794
Salud,34621


In [91]:
## Asignar a los valores nulos "No info" para variable de estrato
df1.FAMI_TIENEINTERNET.values[df.FAMI_TIENEINTERNET.isna()] = 'No info'
sum(df1.FAMI_TIENEINTERNET.isna())
df1['FAMI_TIENEINTERNET'].value_counts()

,count
FAMI_TIENEINTERNET,
Si,592514
No,73357
No info,26629


In [92]:
# Mapeo de valores
map = {
    'Si': 1,
    'No': 0,
    'No info': -1
}

# Aplicar el mapeo a la columna
df1['FAMI_TIENEINTERNET'] = df1['FAMI_TIENEINTERNET'].map(map)

In [93]:
## Asignar a los valores nulos "No info" para variable de estrato
df1.ESTU_PAGOMATRICULAPROPIO.values[df.ESTU_PAGOMATRICULAPROPIO.isna()] = 'No info'
sum(df1.ESTU_PAGOMATRICULAPROPIO.isna())


0

In [94]:
# Mapeo de valores
mapping = {
    'Si': 1,
    'No': 0,
    'No info': -1
}

# Aplicar el mapeo a la columna
df1['ESTU_PAGOMATRICULAPROPIO'] = df1['ESTU_PAGOMATRICULAPROPIO'].map(mapping)


In [95]:
df1['ESTU_PAGOMATRICULAPROPIO'].value_counts()

,count
ESTU_PAGOMATRICULAPROPIO,
0,382201
1,303801
-1,6498


In [96]:
# Se elimina variable respuesta y dos variables categoricas que no se van a usar en el modelo
X_features = df1.drop(['RENDIMIENTO_GLOBAL','PERIODO', 'ESTU_PRGM_ACADEMICO'], axis=1, inplace=False)

In [97]:
X_features.dtypes

,0
ESTU_PRGM_DEPARTAMENTO,object
ESTU_VALORMATRICULAUNIVERSIDAD,float64
ESTU_HORASSEMANATRABAJA,float64
FAMI_ESTRATOVIVIENDA,object
FAMI_TIENEINTERNET,int64
FAMI_EDUCACIONPADRE,object
FAMI_EDUCACIONMADRE,object
ESTU_PAGOMATRICULAPROPIO,int64
AÑO,object
PRGM_AREA,object


In [98]:
# Estrato es categorica ordinal (ya que se puede establecer un orden), por lo tanto se le asigna dicho orden
dic = {'No info':-1, 'Sin Estrato':0, 'Estrato 1':1, 'Estrato 2':2,  'Estrato 3':3,  'Estrato 4':4, 'Estrato 5':5, 'Estrato 6':6}
X_features['FAMI_ESTRATOVIVIENDA'] = X_features['FAMI_ESTRATOVIVIENDA'].replace(dic).infer_objects()


<ipython-input-98-e2d63e75f9bc>:3: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [99]:
X_features['FAMI_ESTRATOVIVIENDA'] = X_features['FAMI_ESTRATOVIVIENDA'].astype('int')

In [100]:
columnas_a_transformar = ['FAMI_EDUCACIONPADRE','FAMI_EDUCACIONMADRE', 'AÑO','PRGM_AREA', 'ESTU_PRGM_DEPARTAMENTO']
columnas_a_escalar = ['FAMI_TIENEINTERNET','ESTU_PAGOMATRICULAPROPIO','ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA']


In [101]:
pd.set_option('display.max_columns', None)

In [102]:
# Dumización de variables independientes de más de 2 categorias
columnas_a_transformar=pd.get_dummies(X_features[columnas_a_transformar])


In [103]:
from sklearn.preprocessing import StandardScaler

X_features[columnas_a_escalar] = StandardScaler().fit_transform(X_features[columnas_a_escalar])


In [104]:
# Seleccionar las columnas no transformadas
escaladas = X_features.drop(columns= ['FAMI_EDUCACIONPADRE','FAMI_EDUCACIONMADRE','AÑO','PRGM_AREA', 'ESTU_PRGM_DEPARTAMENTO'])

# Concatenar las columnas transformadas y las no transformadas
X_features_final = pd.concat([escaladas, columnas_a_transformar], axis=1)

X_features_final.head(3)

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,FAMI_EDUCACIONPADRE_Educación profesional completa,FAMI_EDUCACIONPADRE_Educación profesional incompleta,FAMI_EDUCACIONPADRE_Indeterminado,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta,FAMI_EDUCACIONMADRE_Educación profesional completa,FAMI_EDUCACIONMADRE_Educación profesional incompleta,FAMI_EDUCACIONMADRE_Indeterminado,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta,AÑO_2018,AÑO_2019,AÑO_2020,AÑO_2021,PRGM_AREA_Administracion de empresas,PRGM_AREA_Animales,PRGM_AREA_Arquitectura y Diseño,PRGM_AREA_Ciencias Economicas,PRGM_AREA_Ciencias Politicas,PRGM_AREA_Ciencias Sociales,PRGM_AREA_Derecho,PRGM_AREA_Entretenimiento,PRGM_AREA_Exactas,PRGM_AREA_Ingeniería Industrial,PRGM_AREA_Lenguas,PRGM_AREA_Licenciaturas,PRGM_AREA_Medicina,PRGM_AREA_Negocios,PRGM_AREA_Otras,PRGM_AREA_Otras Admin,PRGM_AREA_Otras Ingenierías,PRGM_AREA_Salud,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_Zonas dificil acceso
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
904256,1.482661,-0.994623,0.485119,0.384334,-0.835923,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
645256,0.191253,-1.340601,0.485119,-1.717718,-0.835923,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
308367,0.191253,1.115843,0.485119,0.384334,-0.835923,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [105]:
# Separar la variable respuesta o dependiente para luego convertir la columna de target (predicción) en valores discretos

y_target = 'RENDIMIENTO_GLOBAL'

rmap = {'alto': 3, 'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2}

# Usar map para transformar los valores
df1[y_target] = df1[y_target].map(rmap)


In [106]:
# Grafico variable respuesta

counts = df1[y_target].value_counts().reset_index()
counts.columns = [y_target, 'Count']  # Renombrar las columnas

counts['Percentage'] = (counts['Count'] / counts['Count'].sum()) * 100
counts['Label'] = counts['Count'].astype(str) + ' (' + counts['Percentage'].round(1).astype(str) + '%)'
print(f'Gráfica')

fig = px.bar(counts, x=y_target, y='Count', color=y_target, title=f'Distribución de {y_target}',
             labels={y_target:'Rendimiento global', 'Count': 'Frecuencia'}, text='Label', width=1000, height=600)

fig.show()

Gráfica


In [107]:
counts

,RENDIMIENTO_GLOBAL,Count,Percentage,Label
0,3,175619,25.360144,175619 (25.4%)
1,0,172987,24.980072,172987 (25.0%)
2,1,172275,24.877256,172275 (24.9%)
3,2,171619,24.782527,171619 (24.8%)


In [108]:
df1[y_target]

,RENDIMIENTO_GLOBAL
ID,
904256,2
645256,0
308367,0
470353,3
989032,1
...,...
25096,2
754213,0
504185,1


In [109]:
y_target

'RENDIMIENTO_GLOBAL'

In [110]:
X_features_final.head(3)

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,FAMI_EDUCACIONPADRE_Educación profesional completa,FAMI_EDUCACIONPADRE_Educación profesional incompleta,FAMI_EDUCACIONPADRE_Indeterminado,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta,FAMI_EDUCACIONMADRE_Educación profesional completa,FAMI_EDUCACIONMADRE_Educación profesional incompleta,FAMI_EDUCACIONMADRE_Indeterminado,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta,AÑO_2018,AÑO_2019,AÑO_2020,AÑO_2021,PRGM_AREA_Administracion de empresas,PRGM_AREA_Animales,PRGM_AREA_Arquitectura y Diseño,PRGM_AREA_Ciencias Economicas,PRGM_AREA_Ciencias Politicas,PRGM_AREA_Ciencias Sociales,PRGM_AREA_Derecho,PRGM_AREA_Entretenimiento,PRGM_AREA_Exactas,PRGM_AREA_Ingeniería Industrial,PRGM_AREA_Lenguas,PRGM_AREA_Licenciaturas,PRGM_AREA_Medicina,PRGM_AREA_Negocios,PRGM_AREA_Otras,PRGM_AREA_Otras Admin,PRGM_AREA_Otras Ingenierías,PRGM_AREA_Salud,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_Zonas dificil acceso
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
904256,1.482661,-0.994623,0.485119,0.384334,-0.835923,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
645256,0.191253,-1.340601,0.485119,-1.717718,-0.835923,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
308367,0.191253,1.115843,0.485119,0.384334,-0.835923,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [111]:
# Ordena las columnas alfabéticamente por nombre.
df1 = df1[sorted(df1.columns)]

# Asigna las características independientes a X
X = X_features_final

# Extrae la columna objetivo y la asigna a la variable y
y = df1[y_target]

#Verificar Forma de X y y
X.shape, y.shape

((692500, 74), (692500,))

In [112]:
y

,RENDIMIENTO_GLOBAL
ID,
904256,2
645256,0
308367,0
470353,3
989032,1
...,...
25096,2
754213,0
504185,1


# <font color=4CBB17>**Modelo XGBOOSTClassifier**</font>


## <font color=FF7F50>**Selección de variables con método Wrapper e Integrado** </font>

In [118]:
# Librerias
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [114]:
# Función recursiva de selección de características
def recursive_feature_selection(X,y,model,k): #model=modelo que me va a servir de estimador
  rfe = RFE(model, n_features_to_select=k, step=1)# step=1 cada cuanto el toma la decision de tomar una caracteristica
  fit = rfe.fit(X, y)
  X_new = fit.support_
  print("Num Features: %s" % (fit.n_features_))
  print("Selected Features: %s" % (fit.support_))
  print("Feature Ranking: %s" % (fit.ranking_))

  return X_new

In [119]:
# Establecer Estimador
model = LogisticRegression() # Se selecionó Regresión Logistica

# Obtener columnas seleciconadas
X_new = recursive_feature_selection(X, y, model, 20)

# Nuevo conjunto de datos
df_newrapper = X.iloc[:,X_new]
df_newrapper.head()

Num Features: 20
Selected Features: [False False False False False  True False False False  True False False
 False False False False False False False  True False  True  True False
 False False False False False False False  True False False  True  True
 False False False  True False  True  True  True False False  True False
  True False False False False False False False False False  True  True
  True False  True False False False False False False False  True False
 False  True]
Feature Ranking: [33 38 25 46 24  1 10 20 13  1 30 29 50 49 39 48 14 27 15  1  3  1  1 53
 51 28 54 31 26 42 18  1 22 32  1  1  2 36 41  1 52  1  1  1 35 23  1 16
  1  6 17  8 34  4  5 37 55 21  1  1  1 40  1 45 43 19 44 12 11  7  1 47
  9  1]


,FAMI_EDUCACIONPADRE_Educación profesional completa,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,PRGM_AREA_Administracion de empresas,PRGM_AREA_Ciencias Economicas,PRGM_AREA_Ciencias Politicas,PRGM_AREA_Exactas,PRGM_AREA_Lenguas,PRGM_AREA_Licenciaturas,PRGM_AREA_Medicina,PRGM_AREA_Otras Admin,PRGM_AREA_Salud,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_Zonas dificil acceso
ID,,,,,,,,,,,,,,,,,,,,
904256,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
645256,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
308367,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
470353,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
989032,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [122]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso

# Selector de variables con Lasso.....se recompueden probar varios alpha....https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html
sel_ = SelectFromModel(Lasso(alpha=0.1, max_iter=10000), max_features=10)  # Se ajusta alpha según sea necesario
sel_.fit(X, y)
print(sel_.estimator_.coef_)

#Obtener variables seleccionadas
X_new = sel_.get_support()#descarta los coeficientes mas cercanos a 0

df_newlasso = X.iloc[:,X_new]
df_newlasso.head()

[ 0.1297692  -0.          0.10890142  0.         -0.07897463  0.
  0.          0.         -0.          0.         -0.         -0.
  0.         -0.          0.         -0.          0.          0.
  0.         -0.          0.         -0.         -0.          0.
 -0.          0.         -0.          0.         -0.          0.
 -0.         -0.         -0.          0.         -0.          0.
 -0.          0.          0.          0.          0.          0.
 -0.          0.          0.         -0.         -0.          0.
 -0.          0.         -0.          0.         -0.          0.
  0.         -0.         -0.         -0.         -0.         -0.
  0.         -0.         -0.         -0.         -0.         -0.
 -0.          0.          0.          0.         -0.         -0.
  0.         -0.        ]


,ESTU_VALORMATRICULAUNIVERSIDAD,FAMI_ESTRATOVIVIENDA,ESTU_PAGOMATRICULAPROPIO
ID,,,
904256,1.482661,0.485119,-0.835923
645256,0.191253,0.485119,-0.835923
308367,0.191253,0.485119,-0.835923
470353,0.836957,1.240860,-0.835923
989032,0.191253,0.485119,-0.835923


In [140]:
# Lo ideal es mantener las categorias completas, por eso, se hace esto con las que arrojo el RFE

# Filtrar columnas con prefijo 'FAMI_EDUCACIONPADRE'
df_padre = X.filter(like='FAMI_EDUCACIONPADRE')

# Filtrar columnas con prefijo 'FAMI_EDUCACIONMADRE'
df_madre = X.filter(like='FAMI_EDUCACIONMADRE')

# Filtrar columnas con prefijo 'ESTU_PRGM_DEPARTAMENTO'
df_departamento = X.filter(like='ESTU_PRGM_DEPARTAMENTO')

# Filtrar columnas con prefijo 'ESTU_PRGM_DEPARTAMENTO'
df_area = X.filter(like='PRGM_AREA')

# Concatenar todas las columnas seleccionadas
df_selected_wrapper = pd.concat([df_padre, df_madre, df_departamento, df_area], axis=1)

df_concat = pd.concat([df_selected_wrapper, df_newlasso], axis=1)
df_concat.head(3)

,FAMI_EDUCACIONPADRE_Educación profesional completa,FAMI_EDUCACIONPADRE_Educación profesional incompleta,FAMI_EDUCACIONPADRE_Indeterminado,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta,FAMI_EDUCACIONMADRE_Educación profesional completa,FAMI_EDUCACIONMADRE_Educación profesional incompleta,FAMI_EDUCACIONMADRE_Indeterminado,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_Zonas dificil acceso,PRGM_AREA_Administracion de empresas,PRGM_AREA_Animales,PRGM_AREA_Arquitectura y Diseño,PRGM_AREA_Ciencias Economicas,PRGM_AREA_Ciencias Politicas,PRGM_AREA_Ciencias Sociales,PRGM_AREA_Derecho,PRGM_AREA_Entretenimiento,PRGM_AREA_Exactas,PRGM_AREA_Ingeniería Industrial,PRGM_AREA_Lenguas,PRGM_AREA_Licenciaturas,PRGM_AREA_Medicina,PRGM_AREA_Negocios,PRGM_AREA_Otras,PRGM_AREA_Otras Admin,PRGM_AREA_Otras Ingenierías,PRGM_AREA_Salud,ESTU_VALORMATRICULAUNIVERSIDAD,FAMI_ESTRATOVIVIENDA,ESTU_PAGOMATRICULAPROPIO
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
904256,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,1.482661,0.485119,-0.835923
645256,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,0.191253,0.485119,-0.835923
308367,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,0.191253,0.485119,-0.835923


In [141]:
df_concat.shape

(692500, 68)

## <font color=FF7F50>**Cross Validation** </font>


In [142]:
from sklearn.model_selection import cross_validate

# Deficición de la función para CV
def cross_validation_accuracy(model, X, y, cv=5):
    """Función para realizar validación cruzada y devolver únicamente métricas de accuracy."""
    results = cross_validate(estimator=model,
                             X=X,
                             y=y,
                             cv=cv,
                             scoring='accuracy',
                             return_train_score=True)

# En este caso solo nos interesa accuracy, pero se pueden agregar mas metricas como recall o F1-score
    return {
        "Training Accuracy scores": results['train_score'],
        "Mean Training Accuracy": results['train_score'].mean() * 100,
        "Validation Accuracy scores": results['test_score'],
        "Mean Validation Accuracy": results['test_score'].mean() * 100
    }

In [144]:
# Uso de la función con un modelo, por ejemplo, un modelo XGBClassifier
from xgboost import XGBClassifier

# Instancia del modelo
model = XGBClassifier(
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    n_estimators=4,
    objective='binary:logistic',
    eval_metric='logloss'
)

# Llamada a la función
cv_results = cross_validation_accuracy(model, df_concat, y)

# Mostrar resultados
print(cv_results)

{'Training Accuracy scores': array([0.38606859, 0.38860469, 0.38667509, 0.38718412, 0.38794043]), 'Mean Training Accuracy': 38.729458483754506, 'Validation Accuracy scores': array([0.38726354, 0.38561011, 0.38807942, 0.38571119, 0.38582671]), 'Mean Validation Accuracy': 38.64981949458484}


## <font color=FF7F50> **5.4. Selección Hiperparámetros : Búsqueda por Cuadrícula**</font>

Ahora se procede a consultar los parámetros que mas influencian en el rendimiento de XGboost, hallando que principalmente, los parametros mas optimizados suelen ser:


*   max_depth
*   learning_rate
*   subsample

Cada parámetro y sus posibles valores los explicamos a continuacion:

*   **max_depth:**La profundidad máxima es un parámetro frecuente entre algoritmos basados en árboles de decisión. Este tiene como meta controlar el máximo de divisiones o ramificaciones hasta las que puede bajar el árbol. Los valores más frecuentes que puede tomar están en el rango de 3-6, usando profundidades de 3 en modelos complejos pero que se desean explicar de manera sencilla y sin mucho sobreajuste. Mientras que los valores de 6 o mayores son porque se desea capturar interacciones más complejas, arriesgándose a sobreajustes.

*   **learning_rate:**El rate de aprendizaje en los modelos de machine learning hace referencia a la velocidad de aprendizaje del modelo. Este tradicionalmente toma valores de 0-1, los cuales se suelen asignar dependiendo de qué tan complejo sea el problema. Optaremos por valores grandes cuando necesitemos una respuesta rápida pero menos precisa y por valores más pequeños para una respuesta más precisa pero lenta de obtener. En nuestro caso, cogimos valores de 1, 0.1, 0.01 y 0.001, los cuales son valores que manejan desde lo más cercano a 1 hasta lo más cercano a 0

*   **subsample:**El subsample es el encargado de determinar con cuántos datos son entrenados los árboles en cada iteración. Este usualmente se usa para disminuir la posibilidad de tener sobreajustes y ayuda a reducir el tiempo que tarda en entrenar el modelo al no usar en cada árbol el 100% de los datos. Este comúnmente suele manejarse en rangos de 0.5-1, números que representan el porcentaje que dedicará el modelo de los datos para entrenar cada árbol.

Referencias

* https://medium.com/@rithpansanga/optimizing-xgboost-a-guide-to-hyperparameter-tuning-77b6e48e289d

* https://blog.dataiku.com/narrowing-the-search-which-hyperparameters-really-matter



In [147]:
from sklearn.model_selection import RandomizedSearchCV

# Define los hiperparametros
params = {
    'max_depth': [4, 5 ,6],
    'learning_rate': [0.1, 0.01, 0.001],
    'subsample': [ 0.6, 0.7, 0.8]
}

# Crea el modelo XGBC
model = XGBClassifier()

# Crea el ramdomSerach
# Definir el RandomizedSearchCV con validación cruzada
random_search = RandomizedSearchCV(estimator = model,
                           param_distributions = params,
                           n_iter=10, # Numero de iteraciones
                           cv = 3,  # Número de folds para validación cruzada
                           scoring = 'accuracy',  # Se puede cambiar por otra métrica si se desea
                           n_jobs = -1,  # Usar todos los núcleos disponibles para acelerar la búsqueda
                           verbose = 2)
# Ajusta el random search
random_search.fit(df_concat, y)

# Imprime set de hiperparametros y el accuracy
print("Best set of hyperparameters: ", random_search.best_params_)
print("Best score: ", random_search.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best set of hyperparameters:  {'subsample': 0.6, 'max_depth': 6, 'learning_rate': 0.1}
Best score:  0.41315090207212735


In [149]:
from xgboost import XGBClassifier

# modelo XGBRegressor
bst = XGBClassifier(**random_search.best_params_) # Tiene por defecto la Lineal en objetive

# Ajustar el modelo a los datos de entrenamiento
bst.fit(df_concat, y)

# Predicciones
preds = bst.predict(df_concat)

#función de cross_validation
model_results_opti = cross_validation_accuracy(bst, df_concat, y, 5)

# Resultados de las métricas de validación cruzada (accuracy)
print("\nTraining Accuracy scores: ", model_results_opti['Training Accuracy scores'])
print("Mean Training Accuracy: ", model_results_opti['Mean Training Accuracy'])
print("Validation Accuracy scores: ", model_results_opti['Validation Accuracy scores'])
print("Mean Validation Accuracy: ", model_results_opti['Mean Validation Accuracy'])


Training Accuracy scores:  [0.42       0.42048736 0.42074368 0.42029422 0.42046209]
Mean Training Accuracy:  42.039747292418774
Validation Accuracy scores:  [0.41432491 0.41104693 0.41418773 0.41262094 0.41284477]
Mean Validation Accuracy:  41.30050541516246
